# I. CÂU HỎI TỰ LUẬN

---

## 1. Word Correction

<div style="border-bottom: 1px dashed white;"></div>

- Xây dựng bộ từ điển từ bộ từ vựng `vocab.txt`

In [4]:
def load_vocab(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    words = sorted(set([line.strip().lower() for line in lines]))
    return words

vocabs = load_vocab(file_path='1-word-correction/vocab.txt')
vocabs

['apple',
 'book',
 'dog',
 'hello',
 'never',
 'please',
 'random',
 'sleep',
 'start',
 'understand']

- Hàm tính khoảng cách `levenshtein_distance`

In [38]:
def levenshtein_distance(token1, token2):
    distances = [[0]*(len(token2)+1) for i in range(len(token1)+1)]

    for t1 in range(len(token1)+1):
        distances[t1][0] = t1
    for t2 in range(len(token2)+1):
        distances[0][t2] = t2

    a, b, c = 0, 0, 0

    for t1 in range(1, len(token1)+1):
        for t2 in range(1, len(token2)+1):
            if token1[t1 - 1] == token2[t2 - 1]:
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - t1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                if a <= b and a >= c:
                    distances[t1][t2] = a + 1
                elif b <= a and b <= c:
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    return distances[len(token1)][len(token2)]

- Giao diện `steamlit`

In [ ]:
import streamlit as st

st.title("Word Correction using Levenshtein Distance")
word = st.text_input('Word:')

if st.button("Compute"):
    # Compute levenshtein distance
    leven_distances = dict()
    for vocab in vocabs:
        leven_distances[vocab] = levenshtein_distance(word, vocab)

    # Sorted by distance
    sorted_distances = dict(
        sorted(leven_distances.items(), key=lambda item: item[1]))
    correct_word = list(sorted_distances.keys())[0]
    st.write('Correct word:', correct_word)

    col1, col2 = st.columns(2)
    col1.write('Vocabulary:')
    col1.write(vocabs)
    col2.write('Distances:')
    col2.write(sorted_distances)

---

## 2. Object Detection

<div style="border-bottom: 1px dashed white;"></div>

- Cài đặt mô hình

In [16]:
import cv2
import numpy as np

MODEL = 'exercise-04/2-object-detection/MobileNetSSD_deploy.caffemodel'
PROTOTXT = 'exercise-04/2-object-detection/MobileNetSSD_deploy.prototxt.txt'

- Tạo hàm `process_image` push input_image vào DNN model để tìm bounding box phù hợp

In [18]:
def process_image(image):
    blob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5
    )
    net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
    net.setInput(blob)
    detections = net.forward()
    return detections

- Tạo hàm `annotate_image` để lọc các bounding box có độ tin cậy thấp

In [ ]:
def annotate_image(
        image, detections, confidence_threshold=0.5
):
    # loop over the detections
    (h, w) = image.shape[:2]
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > confidence_threshold:
            # extract the index of the class label from the 'detectión',
            # then compute the (x, y)-coordinates of the bounding box for the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
            cv2.rectangle(image, (startX, startY), (endX, endY), 70, 2)
    return image

- Giao diện `steamlit`

In [ ]:
import streamlit as st
import numpy as np
from PIL import Image

st.title('Object Detection for Images')
file = st.file_uploader('Upload Image', type=['jpg', 'png', 'jpeg'])
if file is not None:
    st.image(file, caption='Upload Image')
    image = Image.open(file)
    image = np.array(image)
    detections = object_detection.process_image(image)
    processed_image = object_detection.annotate_image(image, detections)
    st.image(processed_image, caption='Processed Image')

---

## 3. Chatbot

<div style="border-bottom: 1px dashed white;"></div>

In [19]:
import streamlit as st
from hugchat import hugchat
from hugchat.login import Login

- Giao diện đăng nhập bên trái

In [ ]:
# App title
st.title('Simple ChatBot')

# Hugging Face Credentials
with st.sidebar:
    st.title('Login HugChat')
    hf_email = st.text_input('Enter E-mail:')
    hf_pass = st.text_input('Enter Password:', type='password')
    if not (hf_email and hf_pass):
        st.warning('Please enter your account!')
    else:
        st.success('Proceed to entering your prompt message!')

- Giao diện Chat

In [ ]:
# Store LLM generated responses
if 'messages' not in st.session_state.keys():
    st.session_state.messages = [
        {'role': 'assistant', 'content': 'How may I help you'}]

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.write(message['content'])

- Tạo hàm `generate_response` để tạo phản hồi từ chatbot dựa trên prompt của user 

In [ ]:
# Function for generating LLM response
def generate_response(prompt_input, email, passwd):
    # Hugging Face Login
    sign = Login(email, passwd)
    # Get cookies 
    cookies = sign.login()
    # Create ChatBot from cookies
    chatbot = hugchat.ChatBot(cookies=cookies.get_dict())
    # Return chatbot's answer with user's prompt_input 
    return chatbot.chat(prompt_input)

# User-provided prompt
if prompt := st.chat_input(disabled=not (hf_email and hf_pass)):
    # Append user's message to session state
    st.session_state.messages.append({'role': 'user', 'content': prompt})
    # Display user's message in chat message format
    with st.chat_message('user'):
        st.write(prompt)

# Generate a new response if last message is not from assistant
if st.session_state.messages[-1]['role'] != 'assistant':
    # Display assistant's message in chat message format
    with st.chat_message('assistant'):
        # Show 'Thinking...' spinner while generating response
        with st.spinner('Thinking...'):
            # Generate response from chatbot
            response = generate_response(prompt, hf_email, hf_pass)
            # Display chatbot's response
            st.write(response)
    # Append assistant's message to session state
    message = {'role': 'assistant', 'content': response}
    st.session_state.messages.append(message)

---

# II. CÂU HỎI TRẮC NGHIỆM

---

In [2]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm

In [3]:
sp = StreamlitPatcher()
sp.jupyter()  # register patcher with streamlit

## Câu 1:

Hàm `st.text` được sử dụng để hiển thị chuỗi văn bản trong giao diện Steamlit

In [10]:
# |exporti

st.text('Hello World!')

```None
Hello World!
```

In [14]:
# |exporti

st.image(r'2-object-detection\image\kitten-dog.jpg')

DeltaGenerator()

In [15]:
# |exporti
options = ['Option 1', 'Option 2', 'Option 3']

selected_option = st.selectbox('Select an option', options)

Dropdown(description='Select an option', options=('Option 1', 'Option 2', 'Option 3'), value='Option 1')

In [26]:
# |exporti

st.slider('Select a value', min_value=0, max_value=100)

0

## Câu 2:

In [32]:
# |exporti
st.multiselect('Your favorite colors:', ['Green', 'Yellow', 'Red', 'Blue'])

options = ['Yellow', 'Red']
st.write('You selected:', options)

SelectMultiple(description='Your favorite colors:', options=('Green', 'Yellow', 'Red', 'Blue'), value=())

You selected:

['Yellow', 'Red']

## Câu 3:

Hàm `st.text_input()` được sử dụng để người dùng nhập văn bản trên giao diện Streamlit

In [36]:
# |exporti
name = st.text_input('Enter your name')
st.write('Your name is:', name)

Textarea(value='', description='Enter your name', placeholder='Type something')

Your name is:

## Câu 4

Hàm `st.image(file_path, caption, width, channel)` được sử dụng để hiển thị hình ảnh trên giao diện Streamlit

In [37]:
# |exporti
st.image(r'2-object-detection\image\kitten-dog.jpg', caption='A cat', width=100, channels='RGB')

DeltaGenerator()

## Câu 5

Khoảng cách chỉnh sửa `leveshtein_distance`

In [39]:
levenshtein_distance('elmets', 'elements')

5

## Câu 6:

Hàm `st.session_state` được sử dụng để chia sẻ các biến trong mỗi phiên truy cập của người dùng

In [42]:
# |exporti
if 'username' not in st.session_state:
    st.text_input('Enter your name', key='username')
else:
    st.write('Hello ', st.session_state['username'])

Textarea(value='', description='Enter your name', placeholder='Type something')

## Câu 7:

In [43]:
# |exporti
with st.form('my_form'):
    col1, col2 = st.columns(2)
    f_name = col1.text_input('First Names')
    l_name = col2.text_input('Last Name')
    submitted = st.form_submit_button('Submit')
    if submitted:
        st.write('First Name: ', f_name, ' Last Name: ', l_name)

## Câu 8:

Hàm `st.file_uploader('Choose files', accpet_multiple_files=True)` được sử dụng để tải lên nhiều file trong giao diện Steamlit

In [44]:
# |exporti
uploađe_files = st.file_uploader('Choose files', accept_multiple_files=True)

## Câu 9:

Hàm `st.code`, `st.echo` và `st.markdown` được sử dụng để hiển thị code trong giao diện Streamlit

In [51]:
code_example = """
def greet(name):
    print(f"Hello, {name}!")
"""

In [52]:
# |exporti
st.code(code_example, language="python")

```python

def greet(name):
    print(f"Hello, {name}!")

```

In [50]:
# |exporti
st.markdown(code_example)


def greet(name):
    print(f"Hello, {name}!")


In [49]:
# |exporti
with st.echo():
    def greet(name):
        print(f"Hello, {name}!")

RuntimeError: generator didn't yield

## Câu 10:

Dung lượng mặc định tối đa mỗi file được tải lên trong Streamlit là `200MB`

Tuy nhiên, có thể tùy chỉnh giá trị này thông qua tham số `max_upload_size` trong cấu hình của ứng dụng :

`streamlit run <tên_file>.py --server.maxUploadSize <giá_trị_dung_lượng>`